In [3]:
import io, os
import numpy as np
from google.cloud import vision
from PIL import ImageTk, Image, ImageDraw
import pandas as pd
import sqlite3
import json

In [168]:
img_path = "static/images/paintings/"
paintings = os.listdir(img_path)

In [ ]:
con = sqlite3.connect('rijksstudio.db')
cur = con.cursor()

In [217]:
def add_object_data(obj, painting, dims):
    normalized_vertices = obj.bounding_poly.normalized_vertices
    
    q = cur.execute("""INSERT OR REPLACE INTO Objects 
               VALUES (?,?,?,?,?,?,?,?);""", (painting, obj.name, obj.score, normalized_vertices[0].x, normalized_vertices[0].y, normalized_vertices[2].x, normalized_vertices[2].y, "HUGGING_FACE"))
    
    con.commit()

In [164]:
# init hugging face model
feature_extractor = DetrFeatureExtractor.from_pretrained("facebook/detr-resnet-50")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")

In [180]:
hugging = []
# vision = []

for painting in paintings:
    objects = get_hugging_objects(painting)
    if len(objects):
        hugging.append(objects)

Reading SK-A-2152.png ...
Objects  ['vase']  found
Reading SK-A-2344.png ...
Objects  ['person']  found
Reading SK-A-385.png ...
Objects  ['skateboard', 'person', 'person', 'person', 'tie', 'person', 'handbag', 'person', 'person', 'person']  found
Reading SK-A-390.png ...
Objects  ['person']  found
Reading SK-A-4052.png ...
Objects  ['dog', 'chair', 'person', 'bowl']  found
Reading SK-A-1857.png ...
Objects  []  found
Reading SK-C-134.png ...
Objects  ['person', 'person', 'person', 'person', 'tie', 'person', 'person', 'person', 'person', 'person', 'person', 'person', 'person']  found
Reading SK-A-4867.png ...
Objects  ['person', 'person', 'person', 'person', 'person', 'horse', 'person', 'person', 'person', 'person', 'horse', 'person', 'person', 'person', 'person']  found
Reading SK-A-4118.png ...
Objects  ['airplane', 'airplane', 'cow', 'person', 'person', 'airplane', 'cow', 'horse', 'airplane', 'person', 'cow', 'cow', 'horse', 'person']  found
Reading SK-A-3930.png ...
Objects  ['wine

In [176]:
def get_hugging_objects(painting):
    print("Reading "+painting+" ...")
    im = Image.open(os.path.join(img_path, painting))

    encoding = feature_extractor(im, return_tensors="pt")
    outputs = model(**encoding)
    
    # keep only predictions of queries with 0.9+ confidence (excluding no-object class)
    probas = outputs.logits.softmax(-1)[0, :, :-1]
    keep = probas.max(-1).values > 0.9
    
    # rescale bounding boxes
    target_sizes = torch.tensor(im.size[::-1]).unsqueeze(0)
    postprocessed_outputs = feature_extractor.post_process(outputs, target_sizes)
    bboxes_scaled = postprocessed_outputs[0]['boxes'][keep]
    
    objects = get_hugging_obj_info(probas[keep], bboxes_scaled, painting)
    print("Objects ",[obj['object_name'] for obj in objects], " found")
    
    return objects

In [163]:
def get_hugging_obj_info(prob, boxes, painting):    
    objects = []
    
    # for object in painting
    for p, (xmin, ymin, xmax, ymax) in zip(prob, boxes.tolist()):
        cl = p.argmax()
        object_name = model.config.id2label[cl.item()]
        box = [xmin, ymin, xmax, ymax]
        
        objects.append({
          'painting_id': painting,
          'object_name': object_name,
          'confidence': p[cl].item(),
          'min_x': xmin,
          'min_y': ymin,
          'max_x': xmax,
          'max_y': ymax
        }) 
    return objects
    

In [181]:
# Save objects into json
json.dump(hugging, open("objects_hugging.json", 'w'))

In [ ]:
# con.close()

hugging face code source

@article{DBLP:journals/corr/abs-2005-12872,
  author    = {Nicolas Carion and
               Francisco Massa and
               Gabriel Synnaeve and
               Nicolas Usunier and
               Alexander Kirillov and
               Sergey Zagoruyko},
  title     = {End-to-End Object Detection with Transformers},
  journal   = {CoRR},
  volume    = {abs/2005.12872},
  year      = {2020},
  url       = {https://arxiv.org/abs/2005.12872},
  archivePrefix = {arXiv},
  eprint    = {2005.12872},
  timestamp = {Thu, 28 May 2020 17:38:09 +0200},
  biburl    = {https://dblp.org/rec/journals/corr/abs-2005-12872.bib},
  bibsource = {dblp computer science bibliography, https://dblp.org}
}